In [1]:
#@title #**1. Install The Required Softwares**

#@markdown Execute this cell first to download and install third-party softwares

from google.colab import files
from IPython.utils import io
from IPython.display import clear_output
import os, re, shutil
import numpy as np, pandas as pd
import tqdm.notebook, subprocess
import concurrent.futures as cf

TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]'

try:
  with tqdm.notebook.tqdm(total = 100, bar_format = TQDM_BAR_FORMAT) as pbar:
    with io.capture_output() as captured:
      %shell pip install zipfile36
      from zipfile36 import ZipFile
      pbar.update(20)

      %shell wget https://vina.scripps.edu/wp-content/uploads/sites/55/2020/12/autodock_vina_1_1_2_linux_x86.tgz
      %shell tar -xzvf autodock_vina_1_1_2_linux_x86.tgz
      %shell cp /content/autodock_vina_1_1_2_linux_x86/bin/vina /usr/bin/vina
      pbar.update(80)
except subprocess.CalledProcessError:
  print(captured)
  raise

  0%|          | 0/100 [elapsed: 00:00 remaining: ?]

In [2]:
#@title #**2. Upload Receptor File**

#@markdown Please upload the receptor file in **PDBQT** format.
#@markdown And you can use the receptor file you obtained from gridbox validation process

#@markdown **Note:** Receptor file name should not contain any space

working_dir = '/content/working'
if (not os.path.isdir(working_dir)):
  os.mkdir(working_dir)
else:
  shutil.rmtree(working_dir)
  os.mkdir(working_dir)
os.chdir(working_dir)

receptor_name = list(files.upload().keys())[0]
clear_output()

In [3]:
#@title #**3. Upload Ligand Files**

#@markdown Please upload PDBQT format-ligand files in **ZIP** archive for simplifying the uploading process.
#@markdown Also keep the ZIP file name away from any "spacing" letter

ligand_dir = os.path.join(working_dir, 'ligand')
if (not os.path.isdir(ligand_dir)):
  os.mkdir(ligand_dir)
else:
  shutil.rmtree(ligand_dir)
  os.mkdir(ligand_dir)

ligand_zip = list(files.upload().keys())[0]
with ZipFile(ligand_zip, 'r') as zf:
  zf.extractall(path = ligand_dir)
clear_output()

os.remove(ligand_zip)

In [4]:
#@title #**4. Specify Gridbox Dimension**

#@markdown Type the center point and size of the gridbox you got from validation process below

#@markdown **Note:** Use a dot (.) for the decimal, not a coma (,)

center_x = 2.7811428595866476 #@param {type:"raw"}
center_y = 64.38310745784214 #@param {type:'raw'}
center_z = 67.97142791748047 #@param {type:'raw'}
size_x = 25 #@param {type:'raw'}
size_y = 25 #@param {type:'raw'}
size_z = 25 #@param {type:'raw'}

In [5]:
#@title #**5. Run Virtual Screening**

#@markdown Run this cell to start virtual screening proses.
#@markdown After the process, an archive file containing files of docked ligands will be downloaded

#@markdown If you are using a google colab pro, enable the check below

colab_pro = False #@param {type:"boolean"}
workers_number = 4
if(colab_pro):
  workers_number = 80

result_dir = os.path.join(working_dir, 'result')
if (not os.path.isdir(result_dir)):
  os.mkdir(result_dir)
else:
  shutil.rmtree(result_dir)
  os.mkdir(result_dir)

def dock(lgn):
  receptor_file = os.path.join(working_dir, receptor_name)
  lgn_file = os.path.join(ligand_dir, lgn)
  result_file = os.path.join(result_dir, lgn)
  result_log_file = os.path.join(result_dir, re.split('\.', lgn)[0] + '.log')

  os.system(f'vina --receptor {receptor_file} --ligand {lgn_file} --center_x {center_x} --center_y {center_y} --center_z {center_z} --size_x {size_x} --size_y {size_y} --size_z {size_z} --out {result_file} --log {result_log_file}')

try:
  with tqdm.notebook.tqdm(total = len(os.listdir(ligand_dir)), bar_format = TQDM_BAR_FORMAT) as pbar:
    with io.capture_output() as captured:
      with cf.ProcessPoolExecutor(max_workers = workers_number) as executor:
        for _ in executor.map(dock, os.listdir(ligand_dir)):
          pbar.update()
except subprocess.CalledProcessError:
  print(captured)
  raise
clear_output()

result_archive = os.path.join(working_dir, 'result.zip')
with ZipFile(result_archive, 'w') as zf:
  for res in os.listdir(result_dir):
    zf.write(os.path.join(result_dir, res), arcname = res)

files.download(result_archive)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>